# Demo environment

This notebook will be used as an introduction to the control and image constructs available within the demo environment of both the robot interfaces and the camera interfaces.

We will cover the steps necessary to connect to and control the robot while also collecting image data from the cameras mounted around and on the robotic arm.

## Camera Access

In this demo we'll be utilizing Intel's [RealSense D435i](https://www.intelrealsense.com/depth-camera-d435i/) for use with out robotic arm. We'll not only want to view these images, but we'll want to collect them as we perform tasks, as well. There any many use cases and approaches one can take when peforming tasks using robotics arms such as these that span the areas of standard supervised, unsupervised, and reinforcement learning. For this reason there will likely be a few combinations of data collection that we may want to employ.

We will start with building the software constructs necessary to view and collect data from the Intel RealSense camera that is connected to the arm of our robot.

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
from ipywidgets import Image, HBox
import IPython
import PIL.Image
from matplotlib import pyplot as plt
import sys
import threading
import time
import math
import traitlets

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

class Camera(traitlets.HasTraits): 
    
    value = traitlets.Any()
    running = traitlets.Bool(default_value=False)
    
    def __init__(self, *args, **kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        
        self.camera_serial_num = None
        
        # Camera pipelines
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        self.config.enable_device(self.cam_serial_num)
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        self.pipeline.start(self.config)
        self._running = False
        
        self.value = None
    
    def stream(self):
        while True:
            self.frames = self.pipeline.wait_for_frames()
            self.depth_frame = self.frames.get_depth_frame()
            self.color_frame = self.frames.get_color_frame()
            depth_image = np.asanyarray(self.depth_frame.get_data())
            color_image = np.asanyarray(self.color_frame.get_data())

            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.3), cv2.COLORMAP_JET)

            self.value = np.hstack((color_image, depth_colormap))
    
    @traitlets.observe('running')
    def view_camera(self, change):
        if change['new'] and not change['old']:
            self._running = True
            thread = threading.Thread(target=self.stream)
            thread.start()
        if change['old'] and not change['new']:
            self._running = False
            # join thread to kill it
            self.thread.join()

In [ ]:
cam_1 = Camera()
cam_1.camera_serial_num = 
image_widget_1 = Image(format='jpeg')

def update_image(change):
    image = change['new']
    image_widget_1.value = bgr8_to_jpeg(image)

In [ ]:
cam_1.running = True
cam_1.observe(update_image, names='value')

camera_link_1 = traitlets.dlink((cam_1, 'value'), (image_widget_1, 'value'), transform=bgr8_to_jpeg)
display(image_widget_1)

In [ ]:
cam_2 = Camera()
image_widget_2 = Image(format='jpeg')

def update_image_2(change):
    image = change['new']
    image_widget_2.value = bgr8_to_jpeg(image)

## Robotic Arm Access

The arm that we are utilizing in this demo is the [ufactory xarm](https://www.xarm.cc/). The company is generous enough to provide an open source python SDK for development against the platform. We will be utilizing this SDK heavily in this demo. It is assumed that the controller being used (in our case an Ubuntu 18.04) already has this SDK installed. If it does not, please follow the installation instructions found [here](https://github.com/xArm-Developer/xArm-Python-SDK#installation)

In [ ]:
from xarm.wrapper import XArmAPI

arm = XArmAPI('192.168.1.244')
arm.motion_enable(enable=True)
arm.set_mode(0)
arm.set_state(state=0)
speed = 50

In [ ]:
arm.set_servo_angle(angle=[0, 0, -20, -10, 0], speed=speed, wait=True)

In [2]:
ctx = rs.context()
for d in ctx.devices:
    print(d)

<pyrealsense2.device: Intel RealSense D435 (S/N: 937422071477)>
